training del tokenizer per migliorare la tokenizzazione del codice python

In [2]:


from datasets import load_dataset
# This can take a few minutes to load, so grab a coffee or tea while you wait!
raw_datasets = load_dataset("code_search_net", "python")


Generating validation split: 100%|██████████| 23107/23107 [00:01<00:00, 20986.77 examples/s]


In [3]:
print(raw_datasets["train"])
print(raw_datasets["train"][123456]["whole_func_string"])

Dataset({
    features: ['repository_name', 'func_path_in_repository', 'func_name', 'whole_func_string', 'language', 'func_code_string', 'func_code_tokens', 'func_documentation_string', 'func_documentation_tokens', 'split_name', 'func_code_url'],
    num_rows: 412178
})
def fetch_genome(genome_id):
    '''Acquire a genome from Entrez

    '''
    # TODO: Can strandedness by found in fetched genome attributes?
    # TODO: skip read/write step?
    # Using a dummy email for now - does this violate NCBI guidelines?
    email = 'loremipsum@gmail.com'
    Entrez.email = email

    print 'Downloading Genome...'
    handle = Entrez.efetch(db='nucleotide', id=str(genome_id), rettype='gb',
                           retmode='text')
    print 'Genome Downloaded...'
    tmpfile = os.path.join(mkdtemp(), 'tmp.gb')
    with open(tmpfile, 'w') as f:
        f.write(handle.read())
    genome = coral.seqio.read_dna(tmpfile)

    return genome


In [4]:
#dal raw text dobbiamo creare una lista di liste di testo.
# questo permette di velocizzare il tokenizer, invece di elaborare un testo alla volta


# Don't uncomment the following line unless your dataset is small!
# Questo codice salverebbe tutto in memoria, ma è enorme
# training_corpus = [raw_datasets["train"][i: i + 1000]["whole_func_string"] for i in range(0, len(raw_datasets["train"]), 1000)]

# se un dataset è enorme, dobbiamo sfruttare la memoria solo per il testo che serve(un po di testo alla volta)
# possiamo risolvere con un python generator

def contatore():
    n = 0
    while True:
        yield n  # Restituisce n e sospende l'esecuzione
        n += 1   # Incrementa n per la prossima chiamata

gen = contatore()  # Creazione del generatore
print(next(gen))  # Output: 0
print(next(gen))  # Output: 1
print(next(gen))  # Output: 2




# questi 2 fanno la stessa cosa.

def get_training_corpus():
    return (
        raw_datasets["train"][i : i + 1000]["whole_func_string"]
        for i in range(0, len(raw_datasets["train"]), 1000)
    )

def get_training_corpus():
    dataset = raw_datasets["train"]
    for start_idx in range(0, len(dataset), 1000):
        samples = dataset[start_idx : start_idx + 1000]
        yield samples["whole_func_string"]               # 


training_corpus = get_training_corpus()

0
1
2


Training a new tokenizer

In [6]:
from transformers import AutoTokenizer
old_tokenizer = AutoTokenizer.from_pretrained("gpt2")


In [9]:
# il nostro tokenizer sarà identico, cambierà solamente il vocabolario
example = '''def add_numbers(a, b):
    """Add the two numbers `a` and `b`."""
    return a + b'''

# spezzettiamo la frase e vediamo che This tokenizer has a few special symbols, like Ġ and Ċ, which denote spaces and newlines, respectively.
tokens = old_tokenizer.tokenize(example)
print(tokens)

['def', 'Ġadd', '_', 'n', 'umbers', '(', 'a', ',', 'Ġb', '):', 'Ċ', 'Ġ', 'Ġ', 'Ġ', 'Ġ"""', 'Add', 'Ġthe', 'Ġtwo', 'Ġnumbers', 'Ġ`', 'a', '`', 'Ġand', 'Ġ`', 'b', '`', '."', '""', 'Ċ', 'Ġ', 'Ġ', 'Ġ', 'Ġreturn', 'Ġa', 'Ġ+', 'Ġb']


In [ ]:
# come possiamo vedere, l'indentazione è gestita come 3 spazi, e non ha senso, ci vuole un carattere aposta. 
# e poi altre cose, tipo la gestione del _ fatta male

# allenniamo il tokenizer da zero!!!
tokenizer = old_tokenizer.train_new_from_iterator(training_corpus, 52000) # 52k è il numero max di tipi di token?

In [15]:
tokens = tokenizer.tokenize(example)
print(tokens)
# nome funzione spezzata su _, nuovi simbli per indentazione e """commento
# infine è più compatto.                          

print(len(tokens))
print(len(old_tokenizer.tokenize(example)))

tokenizer.save_pretrained("code-search-net-tokenizer")
# potrei salvarlo sull'hub hugging-face, facendo account ecc..

['def', 'Ġadd', '_', 'numbers', '(', 'a', ',', 'Ġb', '):', 'ĊĠĠĠ', 'Ġ"""', 'Add', 'Ġthe', 'Ġtwo', 'Ġnumbers', 'Ġ`', 'a', '`', 'Ġand', 'Ġ`', 'b', '`."""', 'ĊĠĠĠ', 'Ġreturn', 'Ġa', 'Ġ+', 'Ġb']
27
36


('code-search-net-tokenizer/tokenizer_config.json',
 'code-search-net-tokenizer/special_tokens_map.json',
 'code-search-net-tokenizer/vocab.json',
 'code-search-net-tokenizer/merges.txt',
 'code-search-net-tokenizer/added_tokens.json',
 'code-search-net-tokenizer/tokenizer.json')

Fatto questo, abbiamo tutto a disposizione per trainare un token da zero per i miei dati, e allenarci un modello da 0 o fare un fine-tuning
Ma ora vediamo in dettaglio cosa fa train_new_from_iterator()


In [25]:
# gestione degli spazi diversi dai diversi tokenizzatori
print(AutoTokenizer.from_pretrained("bert-base-cased")("81s"))
print(AutoTokenizer.from_pretrained("roberta-base").tokenize("81s"))
print(AutoTokenizer.from_pretrained("bert-base-cased")("81s"))
print(AutoTokenizer.from_pretrained("roberta-base")("81s"))
print(AutoTokenizer.from_pretrained("bert-base-cased")("81s").word_ids())
print(AutoTokenizer.from_pretrained("roberta-base")("81s").word_ids())


{'input_ids': [101, 5615, 1116, 102], 'token_type_ids': [0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1]}
['81', 's']
{'input_ids': [101, 5615, 1116, 102], 'token_type_ids': [0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1]}
{'input_ids': [0, 6668, 29, 2], 'attention_mask': [1, 1, 1, 1]}
[None, 0, 0, None]
[None, 0, 1, None]


In [ ]:
# token classification con pipeline
from transformers import pipeline

token_classifier = pipeline("token-classification")
print(token_classifier("My name is Sylvain and I work at Hugging Face in Brooklyn."))

# con aggregation strategy, aggreghiamo i risultati, simple fa la maedia dei risultati di tutti i token della parola
token_classifier = pipeline("token-classification", aggregation_strategy="simple")   
print(token_classifier("My name is Sylvain and I work at Hugging Face in Brooklyn.")) 

No model was supplied, defaulted to dbmdz/bert-large-cased-finetuned-conll03-english and revision 4c53496 (https://huggingface.co/dbmdz/bert-large-cased-finetuned-conll03-english).
Using a pipeline without specifying a model name and revision in production is not recommended.
Some weights of the model checkpoint at dbmdz/bert-large-cased-finetuned-conll03-english were not used when initializing BertForTokenClassification: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight']
- This IS expected if you are initializing BertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Device set to use cuda:0


[{'entity': 'I-PER',
  'score': 0.99938285,
  'index': 4,
  'word': 'S',
  'start': 11,
  'end': 12},
 {'entity': 'I-PER',
  'score': 0.99815494,
  'index': 5,
  'word': '##yl',
  'start': 12,
  'end': 14},
 {'entity': 'I-PER',
  'score': 0.9959072,
  'index': 6,
  'word': '##va',
  'start': 14,
  'end': 16},
 {'entity': 'I-PER',
  'score': 0.99923277,
  'index': 7,
  'word': '##in',
  'start': 16,
  'end': 18},
 {'entity': 'I-ORG',
  'score': 0.9738931,
  'index': 12,
  'word': 'Hu',
  'start': 33,
  'end': 35},
 {'entity': 'I-ORG',
  'score': 0.976115,
  'index': 13,
  'word': '##gging',
  'start': 35,
  'end': 40},
 {'entity': 'I-ORG',
  'score': 0.9887976,
  'index': 14,
  'word': 'Face',
  'start': 41,
  'end': 45},
 {'entity': 'I-LOC',
  'score': 0.9932106,
  'index': 16,
  'word': 'Brooklyn',
  'start': 49,
  'end': 57}]

In [33]:
# Now let’s see how to obtain these results without using the pipeline() function!

# 1 tokenize our input and pass it through the model

from transformers import AutoTokenizer, AutoModelForTokenClassification

model_checkpoint = "dbmdz/bert-large-cased-finetuned-conll03-english"
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)
model = AutoModelForTokenClassification.from_pretrained(model_checkpoint)

example = "My name is Sylvain and I work at Hugging Face in Brooklyn."
inputs = tokenizer(example, return_tensors="pt")
outputs = model(**inputs)
outputs

Some weights of the model checkpoint at dbmdz/bert-large-cased-finetuned-conll03-english were not used when initializing BertForTokenClassification: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight']
- This IS expected if you are initializing BertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


TokenClassifierOutput(loss=None, logits=tensor([[[ 8.7508, -2.2626, -1.5300, -2.2889, -0.6513, -2.0016, -0.0112,
          -2.0860,  0.3335],
         [ 8.4973, -2.3986, -1.3582, -2.7887,  0.7575, -1.8873,  0.4344,
          -1.9900, -0.3397],
         [ 9.4719, -2.2261, -0.9849, -2.6116,  0.1219, -2.0627, -0.1259,
          -1.8758, -0.0609],
         [ 9.8670, -2.2175, -1.3125, -2.4866, -0.2550, -1.8536,  0.0856,
          -1.7520, -0.6437],
         [-0.2011, -2.1873, -1.5316, -2.7110,  8.4025, -2.4168, -0.6980,
          -3.0337, -0.0997],
         [ 0.1065, -2.0520, -1.4787, -2.8139,  7.4525, -2.8399, -0.0626,
          -3.3666, -0.4683],
         [ 0.5985, -2.2538, -1.1926, -3.0111,  7.0070, -2.8675,  0.3492,
          -3.3129, -0.2878],
         [-0.0584, -2.2660, -1.4335, -3.1940,  8.3225, -2.6212, -0.0348,
          -2.9780, -0.2957],
         [ 9.6889, -2.4281, -1.5653, -2.5225, -0.9693, -1.5668,  0.4285,
          -1.9413, -0.6774],
         [ 9.0116, -2.1216, -1.4140, -2.69

In [ ]:
print(inputs["input_ids"].shape)
print(outputs.logits.shape)
# per ogni token, abbiamo 9 valori corrispondente ai logits delle 9 classi/label

torch.Size([1, 19])
torch.Size([1, 19, 9])


In [37]:
# trasformiamo i logits in probabilità(softmax) e poi 
# prendiamo l'id della classe con percentuale più alta

import torch
probabilities = torch.nn.functional.softmax(outputs.logits, dim=-1)[0].tolist()
predictions = outputs.logits.argmax(dim=-1)[0].tolist()
print(predictions)

[0, 0, 0, 0, 4, 4, 4, 4, 0, 0, 0, 0, 6, 6, 6, 0, 8, 0, 0]


In [ ]:
# questa è la mappa id label - 
# I e B servono per dire questo è linizio della parola, questo è il continuo..
# esistono 2 diversi tipi di utilizzo di I e B  
model.config.id2label

{0: 'O',
 1: 'B-MISC',
 2: 'I-MISC',
 3: 'B-PER',
 4: 'I-PER',
 5: 'B-ORG',
 6: 'I-ORG',
 7: 'B-LOC',
 8: 'I-LOC'}

In [41]:
# mapping tra prediction - label e testo
results = []
tokens = inputs.tokens()

for idx, pred in enumerate(predictions):
    label = model.config.id2label[pred]
    if label != "O":
        results.append(
            {"entity": label, "score": probabilities[idx][pred], "word": tokens[idx]}
        )

results

[{'entity': 'I-PER', 'score': 0.9993828535079956, 'word': 'S'},
 {'entity': 'I-PER', 'score': 0.9981549382209778, 'word': '##yl'},
 {'entity': 'I-PER', 'score': 0.9959070682525635, 'word': '##va'},
 {'entity': 'I-PER', 'score': 0.9992326498031616, 'word': '##in'},
 {'entity': 'I-ORG', 'score': 0.9738933444023132, 'word': 'Hu'},
 {'entity': 'I-ORG', 'score': 0.9761150479316711, 'word': '##gging'},
 {'entity': 'I-ORG', 'score': 0.9887976050376892, 'word': 'Face'},
 {'entity': 'I-LOC', 'score': 0.9932106137275696, 'word': 'Brooklyn'}]

In [ ]:
# mancano start e and delle parole! 
# agiungiamo al tokenizer return_offset_mapping
# che ci da inizio e fine del token nella frase

inputs_with_offsets = tokenizer(example, return_offsets_mapping=True)
print(inputs_with_offsets)
example[12:14]


{'input_ids': [101, 1422, 1271, 1110, 156, 7777, 2497, 1394, 1105, 146, 1250, 1120, 20164, 10932, 10289, 1107, 6010, 119, 102], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], 'offset_mapping': [(0, 0), (0, 2), (3, 7), (8, 10), (11, 12), (12, 14), (14, 16), (16, 18), (19, 22), (23, 24), (25, 29), (30, 32), (33, 35), (35, 40), (41, 45), (46, 48), (49, 57), (57, 58), (0, 0)]}


'yl'

In [47]:
# aggiungiamo al codice di prima gli offset

results = []
inputs_with_offsets = tokenizer(example, return_offsets_mapping=True)
tokens = inputs_with_offsets.tokens()
offsets = inputs_with_offsets["offset_mapping"]

for idx, pred in enumerate(predictions):
    label = model.config.id2label[pred]
    if label != "O":
        start, end = offsets[idx]
        results.append(
            {
                "entity": label,
                "score": probabilities[idx][pred],
                "word": tokens[idx],
                "start": start,
                "end": end,
            }
        )

print(results)

[{'entity': 'I-PER', 'score': 0.9993828535079956, 'word': 'S', 'start': 11, 'end': 12}, {'entity': 'I-PER', 'score': 0.9981549382209778, 'word': '##yl', 'start': 12, 'end': 14}, {'entity': 'I-PER', 'score': 0.9959070682525635, 'word': '##va', 'start': 14, 'end': 16}, {'entity': 'I-PER', 'score': 0.9992326498031616, 'word': '##in', 'start': 16, 'end': 18}, {'entity': 'I-ORG', 'score': 0.9738933444023132, 'word': 'Hu', 'start': 33, 'end': 35}, {'entity': 'I-ORG', 'score': 0.9761150479316711, 'word': '##gging', 'start': 35, 'end': 40}, {'entity': 'I-ORG', 'score': 0.9887976050376892, 'word': 'Face', 'start': 41, 'end': 45}, {'entity': 'I-LOC', 'score': 0.9932106137275696, 'word': 'Brooklyn', 'start': 49, 'end': 57}]


In [50]:
# ora raggruppiamo le entità ed eliminiamo quelle inutili
# per fare ciò utilizzaimo B e I
import numpy as np

results = []
inputs_with_offsets = tokenizer(example, return_offsets_mapping=True)
tokens = inputs_with_offsets.tokens()
offsets = inputs_with_offsets["offset_mapping"]

idx = 0
while idx < len(predictions):
    pred = predictions[idx]
    label = model.config.id2label[pred]
    if label != "O":
        # Remove the B- or I-
        label = label[2:]
        start, _ = offsets[idx]

        # Grab all the tokens labeled with I-label
        all_scores = []
        while (
            idx < len(predictions)
            and model.config.id2label[predictions[idx]] == f"I-{label}"
        ):
            all_scores.append(probabilities[idx][pred])
            _, end = offsets[idx]
            idx += 1

        # The score is the mean of all the scores of the tokens in that grouped entity
        score = np.mean(all_scores).item()
        word = example[start:end]
        results.append(
            {
                "entity_group": label,
                "score": score,
                "word": word,
                "start": start,
                "end": end,
            }
        )
    idx += 1

results

[{'entity_group': 'PER',
  'score': 0.9981693774461746,
  'word': 'Sylvain',
  'start': 11,
  'end': 18},
 {'entity_group': 'ORG',
  'score': 0.9796019991238912,
  'word': 'Hugging Face',
  'start': 33,
  'end': 45},
 {'entity_group': 'LOC',
  'score': 0.9932106137275696,
  'word': 'Brooklyn',
  'start': 49,
  'end': 57}]

### Normalization and pre-tokenization

In [60]:
# normalization è quel processo che può togliere spazi, accenti, maiuscole ecc.
# alcuni tokenizer possono essere ok, altri no.
# con backend_tokenizer posso avere accesso a questo step intermedio

from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")
print(type(tokenizer.backend_tokenizer))
print(tokenizer.backend_tokenizer.normalizer.normalize_str("Héllò hôw are    ü?"))

tokenizer = AutoTokenizer.from_pretrained("bert-base-cased")
print(tokenizer.backend_tokenizer.normalizer.normalize_str("Héllò hôw are  ü?"))

<class 'tokenizers.Tokenizer'>
hello how are    u?
Héllò hôw are  ü?


In [ ]:
# pre-tokenization aggiunge caratteri speciali (_ per stzio o G^ per inizio parole) e poi divide
# anche qui ogni tokenization lo fa a suo modo

tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")
print(tokenizer.backend_tokenizer.pre_tokenizer.pre_tokenize_str(
    tokenizer.backend_tokenizer.normalizer.normalize_str("Héllò hôw are    ü?"))
    )

tokenizer = AutoTokenizer.from_pretrained("bert-base-cased")
print(tokenizer.backend_tokenizer.pre_tokenizer.pre_tokenize_str("Hello, how are  you?"))

tokenizer = AutoTokenizer.from_pretrained("gpt2")
print(tokenizer.backend_tokenizer.pre_tokenizer.pre_tokenize_str("Hello, how are  you?"))

tokenizer = AutoTokenizer.from_pretrained("t5-small")
print(tokenizer.backend_tokenizer.pre_tokenizer.pre_tokenize_str("Hello, how are  you?")) # non tiene conto degli spazi multipli

# il risultato è la divisione e il mapping delle posizioni. i doppi spazi ad esempio si ricostruiscono grazie a ciò 


[('hello', (0, 5)), ('how', (6, 9)), ('are', (10, 13)), ('u', (17, 18)), ('?', (18, 19))]
[('Hello', (0, 5)), (',', (5, 6)), ('how', (7, 10)), ('are', (11, 14)), ('you', (16, 19)), ('?', (19, 20))]
[('Hello', (0, 5)), (',', (5, 6)), ('Ġhow', (6, 10)), ('Ġare', (10, 14)), ('Ġ', (14, 15)), ('Ġyou', (15, 19)), ('?', (19, 20))]


[('▁Hello,', (0, 6)),
 ('▁how', (7, 10)),
 ('▁are', (11, 14)),
 ('▁you?', (16, 20))]

### 3 subwords tokenization algorythm: BPE, WordPiece, Unigram
### spiegati nel report. 


In [ ]:
## Building a new tokenizer

# carico il dataset, questo è un piccolo dataset in inglese
from datasets import load_dataset

dataset = load_dataset("wikitext", name="wikitext-2-raw-v1", split="train")

def get_training_corpus(): # generator che ti da 1000 sentence alla volta (batch). 
    for i in range(0, len(dataset), 1000):
        yield dataset[i : i + 1000]["text"]


# al posto di scaricare il dataset, si potrebbe voler utilizzare un file di testo. 
with open("wikitext-2.txt", "w", encoding="utf-8") as f:
    for i in range(len(dataset)):
        f.write(dataset[i]["text"] + "\n")

/home/apetrella/miniconda3/envs/llm/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Generating validation split: 100%|██████████| 3760/3760 [00:00<00:00, 1034000.99 examples/s]


In [ ]:
# building a Wordpiece tokenizer from scratch (quello di BErt PRECISAMENTE.)

from tokenizers import (
    decoders,
    models,
    normalizers,
    pre_tokenizers,
    processors,
    trainers,
    Tokenizer,
)

tokenizer = Tokenizer(models.WordPiece(unk_token="[UNK]"))


In [6]:
# normalizer

# tokenizer.normalizer = normalizers.BertNormalizer(lowercase=True) # questa riga basta a replicare bertbase uncased tokenizer

# questo fa la stessa cosa ma a mano
# toglie accenti, toglie caratteri di controllo
# lowercase, 
tokenizer.normalizer = normalizers.Sequence(
    [normalizers.NFD(), normalizers.Lowercase(), normalizers.StripAccents()]   # questo fa la stessa cosa ma a mano
)
print(tokenizer.normalizer.normalize_str("Héllò hôw are ü?"))

hello how are u?


In [9]:
# pretokenization

# SPLITTA SU whitespace and all characters that are not letters, digits, or the underscore characte

tokenizer.pre_tokenizer = pre_tokenizers.BertPreTokenizer()   # pre built
tokenizer.pre_tokenizer = pre_tokenizers.Whitespace()      # a mano
print(tokenizer.pre_tokenizer.pre_tokenize_str("Let's test my pre-tokenizer."))

#se vuoi splittare solo su spazi
pre_tokenizer_spazi = pre_tokenizers.WhitespaceSplit()
print(pre_tokenizer_spazi.pre_tokenize_str("Let's test my pre-tokenizer."))


[('Let', (0, 3)), ("'", (3, 4)), ('s', (4, 5)), ('test', (6, 10)), ('my', (11, 13)), ('pre', (14, 17)), ('-', (17, 18)), ('tokenizer', (18, 27)), ('.', (27, 28))]
[("Let's", (0, 5)), ('test', (6, 10)), ('my', (11, 13)), ('pre-tokenizer.', (14, 28))]


In [15]:
# trainare il tokenizer
special_tokens = ["[UNK]", "[PAD]", "[CLS]", "[SEP]", "[MASK]"]
trainer = trainers.WordPieceTrainer(vocab_size=25000, special_tokens=special_tokens) # possiamo speicificare anche il minimo della frequenza dei toke e alternative a ##

tokenizer.train_from_iterator(get_training_corpus(), trainer=trainer)  # potevamo usare il file di testo
encoding = tokenizer.encode("Let's test this tokenizer.")
print(encoding.tokens)
print(encoding.ids)





['let', "'", 's', 'test', 'this', 'tok', '##eni', '##zer', '.']
[3019, 11, 61, 3611, 1519, 24318, 18903, 6614, 18]
